In [1]:
import sys
sys.path.append('..')

import tensorflow as tf

from pathlib import Path
from tqdm import tqdm

from benchmark.movie_recommender import MovieRecommender

2023-12-03 22:02:22.099223: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-03 22:02:22.134350: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 22:02:22.134388: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 22:02:22.135428: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 22:02:22.143275: I tensorflow/core/platform/cpu_feature_guar

In [2]:
model_path = str('../models/best/DCN.ckpt')
movie_rec = MovieRecommender(model_path)

In [3]:
import tensorflow_datasets as tfds

ratings = tfds.load("movielens/100k-ratings", split="train")
ratings = ratings.map(lambda x: {
    "user_id": x["user_id"],
    "movie_title": x["movie_title"],
})

# train/test split is preserved due to random seed
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)
test = shuffled.skip(80_000).take(20_000)
test = tfds.as_dataframe(test)
test.head()

,movie_title,user_id
0,b'M*A*S*H (1970)',b'346'
1,b'Volcano (1997)',b'602'
2,b'2001: A Space Odyssey (1968)',b'393'
3,b'Dances with Wolves (1990)',b'152'
4,b'Speed (1994)',b'738'


In [4]:
test = test.copy().groupby('user_id', as_index=False)['movie_title'].agg({'actual': (lambda x: list(set(x)))})
test = test.set_index("user_id")
test

,actual
user_id,
b'1',"[b'Birdcage, The (1996)', b'Flipper (1996)', b..."
b'10',"[b'Top Gun (1986)', b'GoodFellas (1990)', b'Ra..."
b'100',"[b'Jackie Brown (1997)', b'Big Bang Theory, Th..."
b'101',"[b'Ransom (1996)', b'Twister (1996)', b'Fright..."
b'102',"[b'Nightmare Before Christmas, The (1993)', b'..."
...,...
b'95',"[b'Ransom (1996)', b'Dangerous Minds (1995)', ..."
b'96',"[b'Fish Called Wanda, A (1988)', b'Die Hard (1..."
b'97',"[b'Stand by Me (1986)', b'Fish Called Wanda, A..."


In [5]:
# make recommendations for all members in the test data
dcn_recs = []
for user in tqdm(test.index):
    _, dcn_predictions = movie_rec.predict(int(user), recommend_seen=True, k=10)
    dcn_recs.append(dcn_predictions)

test['dcn_pred'] = dcn_recs
test.head()

  0%|                                                                                                                                                                                      | 0/942 [00:00<?, ?it/s]2023-12-03 22:03:01.870684: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2023-12-03 22:03:01.871054: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. 

,actual,dcn_pred
user_id,,
b'1',"[b'Birdcage, The (1996)', b'Flipper (1996)', b...","[b'Mad Love (1995)', b'Higher Learning (1995)'..."
b'10',"[b'Top Gun (1986)', b'GoodFellas (1990)', b'Ra...","[b'Substance of Fire, The (1996)', b'Substance..."
b'100',"[b'Jackie Brown (1997)', b'Big Bang Theory, Th...","[b'Other Voices, Other Rooms (1997)', b'Full S..."
b'101',"[b'Ransom (1996)', b'Twister (1996)', b'Fright...","[b'Fear (1996)', b'Fan, The (1996)', b'First K..."
b'102',"[b'Nightmare Before Christmas, The (1993)', b'...","[b'Audrey Rose (1977)', b'Turbulence (1997)', ..."


In [24]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [27]:
import recmetrics

actual = test.actual.values.tolist()
dcn_pred = test.dcn_pred.values.tolist()

print(f'MAP@1:\t{mapk(actual, dcn_pred, k=1)}')
print(f'MAR@1:\t{recmetrics.mark(actual, dcn_pred, k=1)}')
print(f'MAP@5:\t{mapk(actual, dcn_pred, k=5)}')
print(f'MAR@5:\t{recmetrics.mark(actual, dcn_pred, k=5)}')
print(f'MAP@10:\t{mapk(actual, dcn_pred, k=10)}')
print(f'MAR@10:\t{recmetrics.mark(actual, dcn_pred, k=10)}')

MAP@1:	0.006369426751592357
MAR@1:	0.0012060627506487378
MAP@5:	0.006340233545647558
MAR@5:	0.0034913429306858796
MAP@10:	0.0077628714161610225
MAR@10:	0.00620951802509859
